In [1]:
import torch
import torchvision.transforms.functional as F
from src import model, dataset, augmentations

In [2]:
class Resize(augmentations.T.Resize):
    def forward(self, image, mask):
        image = F.resize(image, self.size, self.interpolation)
        mask = F.resize(mask, self.size, self.interpolation)
        return (image,), (mask,)

In [3]:
dataset = dataset.CracksDataset('valid')
# dataset.transform = augmentations.Compose([
#     Resize(224),
#     augmentations.ToTensor(),
# ])
loader = dataset.get_loader(batch_size=2, num_workers=0)
batch = next(iter(loader))

In [4]:
net = model.UNetClassifier().eval()
state_dict_segmentation = torch.load('logs/segmentation/checkpoints/best.pth', map_location='cpu')['model_state_dict']
net.load_state_dict(state_dict_segmentation, strict=False)
state_dict_classification = torch.load('logs/classification/checkpoints/best.pth', map_location='cpu')['model_state_dict']
net.load_state_dict(state_dict_classification, strict=False)

_IncompatibleKeys(missing_keys=['decoder.deconv4.weight', 'decoder.deconv4.bias', 'decoder.conv4.0.weight', 'decoder.conv4.1.weight', 'decoder.conv4.1.bias', 'decoder.conv4.1.running_mean', 'decoder.conv4.1.running_var', 'decoder.conv4.3.weight', 'decoder.conv4.4.weight', 'decoder.conv4.4.bias', 'decoder.conv4.4.running_mean', 'decoder.conv4.4.running_var', 'decoder.deconv3.weight', 'decoder.deconv3.bias', 'decoder.conv3.0.weight', 'decoder.conv3.1.weight', 'decoder.conv3.1.bias', 'decoder.conv3.1.running_mean', 'decoder.conv3.1.running_var', 'decoder.conv3.3.weight', 'decoder.conv3.4.weight', 'decoder.conv3.4.bias', 'decoder.conv3.4.running_mean', 'decoder.conv3.4.running_var', 'decoder.deconv2.weight', 'decoder.deconv2.bias', 'decoder.conv2.0.weight', 'decoder.conv2.1.weight', 'decoder.conv2.1.bias', 'decoder.conv2.1.running_mean', 'decoder.conv2.1.running_var', 'decoder.conv2.3.weight', 'decoder.conv2.4.weight', 'decoder.conv2.4.bias', 'decoder.conv2.4.running_mean', 'decoder.conv2.

In [5]:
targets = batch.pop('masks')
inputs = batch.pop('images')
outputs, classes = net(inputs)
print(classes)

tensor([[0.8677],
        [1.0000],
        [1.0000],
        [0.9992],
        [1.0000],
        [0.9969],
        [0.6887],
        [1.0000],
        [1.0000],
        [1.0000]], grad_fn=<SigmoidBackward>)


In [6]:
save_fmt = 'tests/{}.png'
for index, (inp, tar, out) in enumerate(zip(inputs, targets, outputs)):
    F.to_pil_image(inp).save(save_fmt.format(f'input_{index}'))
    F.to_pil_image(tar).save(save_fmt.format(f'target_{index}'))
    F.to_pil_image(out).save(save_fmt.format(f'predict_{index}'))
    # print(inp.shape, tar.shape, out.shape)

In [ ]:
import torch
from src import model
net = model.UNetClassifier()
inputs1 = torch.rand(3, 224, 224)
inputs2 = torch.randn(3, 224, 224)
inputs = torch.stack((inputs1, inputs2))
masks, classes = net(inputs)

In [ ]:
masks[1].sum()

In [ ]:
torch.where(classes > 0.5)

In [ ]:
idx1

In [ ]:
import torch
filepath = 'logs/segmentation/checkpoints/last.pth'
state_dict = torch.load(filepath)
model_state_dict = state_dict['model_state_dict'].copy()
print(model_state_dict.keys())

In [ ]:
new_model_state_dict = {}
for key, value in model_state_dict.items():
    new_model_state_dict[key.replace('VGGEncoder', 'encoder')] = value

In [ ]:
state_dict['model_state_dict'] = new_model_state_dict

In [ ]:
torch.save(state_dict, filepath)

In [31]:
from sympy import *
y = Symbol('y')
p = 1 / (1 + exp(-y))
# p = Function('p')(y)

t = Symbol('t')
f = 2 * p * t + 1
g = p + t + 1
h = 1 - f / g
dh = Derivative(h, y).doit()
dh

-2*t*exp(-y)/((1 + exp(-y))**2*(t + 1 + 1/(1 + exp(-y)))) + (2*t/(1 + exp(-y)) + 1)*exp(-y)/((1 + exp(-y))**2*(t + 1 + 1/(1 + exp(-y)))**2)

In [39]:
dh.subs({'y': 10, 't': 0}).evalf()

1.13494671697606e-5

In [40]:
dh.subs({'y': -10, 't': 1}).evalf()

-3.40453101998998e-5